In [ ]:
# @title Initialize
hf_token = "" # @param {"type":"string","placeholder":"Your HuggingFace access token"}

from pathlib import Path
Path("/content/images").mkdir(exist_ok=True)
Path("/content/ffmpeg").mkdir(exist_ok=True)

!git clone --branch cuda https://github.com/hhkcu/open-oasis.git

print("Installing dependencies")

%cd /content

!wget -q "$(curl -s -I https://github.com/cloudflare/cloudflared/releases/latest|grep location|cut -d" " -f 2|tr -d "\r"|sed s/tag/download/)/cloudflared-linux-amd64" -O /content/cloudflared
!chmod a+x /content/cloudflared
!wget "https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz" -O ffmpeg.tar.xz
!tar xf ffmpeg.tar.xz --strip-components=1 -C /content/ffmpeg

!pip install -U "huggingface-hub[cli]"
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install einops diffusers timm av websockets ffmpeg-python safetensors

%cd /content/open-oasis

print("Downloading models")

!huggingface-cli login --token $hf_token
!huggingface-cli download --local-dir . Etched/oasis-500m oasis500m.safetensors
!huggingface-cli download --local-dir . Etched/oasis-500m vit-l-20.safetensors


To insert a custom starting image, run the cell above to initialize everything, then place any images you want into the `images` folder.

After that, run the cell below.

In [ ]:
# @title Parse Starting Images
import os, pathlib, ffmpeg, sys

crop_width=640
crop_height=360

imagesPath = "/content/images"
samplesPath = os.path.join("/content/open-oasis", "sample_data")
for fn in os.listdir(imagesPath):
  print(os.path.join(samplesPath, pathlib.Path(fn).stem+".mp4"))
  ffmpeg.input(os.path.join(imagesPath, fn),loop=1).output(os.path.join(samplesPath, pathlib.Path(fn).stem+".mp4"),vcodec="libx264",t=1,pix_fmt="yuv420p",vf=f'crop=iw:iw*9/16,scale={crop_width}:{crop_height}').run()

In [ ]:
# @title Run Game
starting_img_name = "2024-11-04_20.07.59" # @param {"type":"string","placeholder":"Filename of the starting image"}
%cd /content
!./cloudflared tunnel --url http://127.0.0.1:17890 &>c.log &
!sleep 6
!grep "  https" c.log |cut -d " " -f 5
%cd /content/open-oasis
!STARTING_IMAGE_NAME=$starting_img_name python3 game.py


In [29]:
# @title fix
!pkill cloudflared

In [ ]:
# @title (for me) pull
%cd /content/open-oasis
!git pull